In [203]:
# Adapted from https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/notebooks/tensorflow/getting-started-keras.ipynb
import os
import sys

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials/mnist-keras-mle-gcs-access.json'


In [187]:
# Test that the package works
! python3 -m trainer.task \
    --job-dir ./train-output

^C
Traceback (most recent call last):
  File "/Users/julian/projects/mnist-keras-gcp/.venv/lib/python3.7/site-packages/numpy/core/__init__.py", line 24, in <module>
    from . import multiarray
  File "/Users/julian/projects/mnist-keras-gcp/.venv/lib/python3.7/site-packages/numpy/core/multiarray.py", line 14, in <module>
    from . import overrides
  File "/Users/julian/projects/mnist-keras-gcp/.venv/lib/python3.7/site-packages/numpy/core/overrides.py", line 7, in <module>
    from numpy.core._multiarray_umath import (
ImportError: PyCapsule_Import could not import module "datetime"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code,

In [25]:
# Explicitly tell `gcloud ai-platform local train` to use Python 3 
! gcloud config set ml_engine/local_python $(which python3)

# Run the trainer with ai-platform local train
! gcloud ai-platform local train \
    --package-path ./trainer \
    --module-name trainer.task \
    --job-dir ./train-output

Updated property [ml_engine/local_python].
2020-04-21 19:10:12.090005: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-21 19:10:12.105380: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ffae95fe850 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-21 19:10:12.105423: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1500 steps, validate on 375 steps
^C


In [193]:
import datetime
os_variables = {
    'BUCKET_NAME': 'mnist-keras-mle',
    'REGION': 'europe-west1',
    'JOB_NAME': 'mynet_job',
    'JOB_DIR': 'gs://' + BUCKET_NAME + '/training-jobs'
}
for key, value in os_variables.items():
    os.environ[key] = value

! export date=$(date +%y%m%d_%H%M%S); \
  gcloud ai-platform jobs submit training "${JOB_NAME}_${date}" \
    --config hptuning_config.yaml \
    --package-path trainer/ \
    --module-name trainer.task \
    --region $REGION \
    --python-version 3.7 \
    --runtime-version 2.1 \
    --job-dir "${JOB_DIR}/${date}" \
    -- \
    --epochs 5

Job [mynet_job_200422_170907] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe mynet_job_200422_170907

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs mynet_job_200422_170907
jobId: mynet_job_200422_170907
state: QUEUED


In [ ]:
# view training logs
! tensorboard --logdir $JOB_DIR

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.1.1 at http://localhost:6006/ (Press CTRL+C to quit)


In [200]:
MODEL_NAME="mynet"

! gcloud ai-platform models create $MODEL_NAME \
  --regions $REGION

Created ml engine model [projects/mnist-keras-mle/models/mynet].


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



In [211]:
MODEL_VERSION="v1"

# Get a list of directories in the `keras_export` parent directory. Then pick
# the directory with the latest timestamp, in case you've trained multiple
# times.
SAVED_MODEL_PATH=$(gsutil ls $JOB_DIR/keras_export | tail -n 1)

# Create model version based on that SavedModel directory
gcloud ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --runtime-version 1.15 \
  --python-version 3.7 \
  --framework tensorflow \
  --origin $SAVED_MODEL_PATH